In [1]:
import pandas as pd


In [3]:
#load movie data
movies = pd.read_csv('raw/data.tsv', sep='\t')
movies = movies[movies.titleType == 'movie']
movies = movies[['tconst', 'titleType', 'originalTitle','startYear', 'genres']]
movies['genres'] .unique

C:\Users\luisa\AppData\Local\Temp\ipykernel_1720\3034390900.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('data.tsv', sep='\t')


In [7]:
selected_movies =(movies[['tconst', 'titleType', 'originalTitle','startYear']]).join(movies.genres.str.strip(',').str.split(',').str[0].str.get_dummies()[['Drama', 'Comedy', 'Romance', 'Thriller']])
selected_mov = selected_movies[~((selected_movies.Drama == 0) & (selected_movies.Comedy == 0) & (selected_movies.Romance == 0) & (selected_movies.Thriller == 0))]
selected_mov

In [11]:
mask = selected_mov.startYear.str.len() == 4
selected_mov = selected_mov.loc[mask]
selected_mov

In [13]:
filtered_dfs = []
for column in selected_mov[['Drama', 'Comedy','Romance','Thriller']]:
    ones_indices = selected_mov[selected_mov[column] == 1].head(500).index
    filtered_df = selected_mov.loc[ones_indices]
    filtered_dfs.append(filtered_df)
final_filtered_df = pd.concat(filtered_dfs, ignore_index=True)
list_movie = list(final_filtered_df.tconst)

In [16]:
#load actor data
act = pd.read_csv('data_2.tsv', sep='\t')

In [17]:
act.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0045537"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0075213"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0056404,tt0057345,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0077975,tt0080455,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0069467,tt0083922,tt0050986"


In [18]:
act2 =(act[['nconst', 'primaryName', 'knownForTitles']]).join(act.primaryProfession.str.strip(',').str.split(',').str[0].str.get_dummies())

In [19]:
act3 = act2[['nconst', 'actor','primaryName','actress','knownForTitles','director']]
act3[act3.director==1]

,nconst,actor,primaryName,actress,knownForTitles,director
32,nm0000033,0,Alfred Hitchcock,0,"tt0052357,tt0056869,tt0053125,tt0054215",1
39,nm0000040,0,Stanley Kubrick,0,"tt0066921,tt0120663,tt0072684,tt0062622",1
87,nm0000088,0,Aleksey Korenev,0,"tt0068304,tt0191104,tt0149586,tt0100069",1
90,nm0000091,0,Gérard Pirès,0,"tt0064297,tt0421974,tt0282552,tt0152930",1
230,nm0000231,0,Oliver Stone,0,"tt0110632,tt0096969,tt0102138,tt0091763",1
...,...,...,...,...,...,...
12765380,nm9993420,0,Viorel Damian,0,tt8743232,1
12765400,nm9993446,0,Adriana Gilma,0,tt8743296,1
12765579,nm9993645,0,Fotis Georgopoulos,0,tt8743742,1
12765608,nm9993679,0,Art Jones,0,tt8744074,1


In [20]:
act3 = act3[act3.knownForTitles.str.len() > 3]

In [21]:
act3  =act3[~((act3.actor == 0 ) & (act3.actress == 0) & (act3.director == 0))]
act3[act3.director==1]

,nconst,actor,primaryName,actress,knownForTitles,director
32,nm0000033,0,Alfred Hitchcock,0,"tt0052357,tt0056869,tt0053125,tt0054215",1
39,nm0000040,0,Stanley Kubrick,0,"tt0066921,tt0120663,tt0072684,tt0062622",1
87,nm0000088,0,Aleksey Korenev,0,"tt0068304,tt0191104,tt0149586,tt0100069",1
90,nm0000091,0,Gérard Pirès,0,"tt0064297,tt0421974,tt0282552,tt0152930",1
230,nm0000231,0,Oliver Stone,0,"tt0110632,tt0096969,tt0102138,tt0091763",1
...,...,...,...,...,...,...
12765380,nm9993420,0,Viorel Damian,0,tt8743232,1
12765400,nm9993446,0,Adriana Gilma,0,tt8743296,1
12765579,nm9993645,0,Fotis Georgopoulos,0,tt8743742,1
12765608,nm9993679,0,Art Jones,0,tt8744074,1


In [22]:
act3 = act3[act3.knownForTitles.isin(list_movie)]

In [23]:
tog = act3.merge(selected_mov, right_on='tconst', left_on='knownForTitles', how='outer').dropna()

In [24]:
gen = tog[['Drama', 'Comedy', 'Romance', 'Thriller']]
new = gen.dot(gen.columns)
tog['genres'] = new
tog[['actor','actress','director']] = tog[['actor','actress','director']].astype(int)
job = tog[['actor','actress','director']]
tog['job'] = job.dot(job.columns)
tog

,nconst,actor,primaryName,actress,knownForTitles,director,tconst,titleType,originalTitle,startYear,Drama,Comedy,Romance,Thriller,genres,job
0,nm0007562,1,Imad Ab on Khaled,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
1,nm0196878,1,Emad Daher,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
2,nm0251614,1,Husein Abu Eid,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
3,nm0259524,1,Hans Erlandsson,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
4,nm0323487,1,Ibrahim Gneim,0,tt0098544,0,tt0098544,movie,Täcknamn Coq Rouge,1989,0,0,0,1,Thriller,actor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3087,nm9324389,1,Gunnel Holzhausen,0,tt0002514,0,tt0002514,movie,De svarta maskerna,1912,1,0,0,0,Drama,actor
3088,nm9507151,0,Frl. Stoike,1,tt0001914,0,tt0001914,movie,Die Sünden der Väter,1913,1,0,0,0,Drama,actress
3089,nm9843327,1,N. Moroz,0,tt0083794,0,tt0083794,movie,Davay pozhenimsya,1982,0,0,1,0,Romance,actor
3090,nm9843328,1,V. Kalashnikov,0,tt0083794,0,tt0083794,movie,Davay pozhenimsya,1982,0,0,1,0,Romance,actor


In [25]:
grouped = tog.groupby('tconst')
for name,group in grouped:
    print(group.index[0])

726
1504
1675
2691
1498
2356
2210
2948
388
2547
2956
1448
2631
3088
359
1357
2842
2942
697
1491
2584
952
1707
3087
2786
2208
2947
1972
2806
2880
1792
2930
2701
2477
2373
2881
2706
791
1312
2023
1456
3006
2580
591
1768
2222
595
2435
1585
2343
679
1307
2992
2712
2125
2364
1273
1739
2071
2196
1745
1826
703
2018
2361
2062
2233
1330
1406
2305
1775
2120
1556
852
870
1809
1542
2126
2342
1084
2044
2029
557
904
2106
2123
2943
1451
2695
1596
3025
574
2530
1718
1586
2505
2473
2491
1818
1041
702
2064
1663
1046
2192
1061
2177
1600
1871
1429
2300
2371
2256
2008
2365
1885
1035
119
1345
2431
1815
1201
391
1054
2359
1042
1310
2224
315
2017
2212
2163
885
1719
2889
1536
2228
1591
845
1761
1856
1662
2344
1755
120
1032
471
1659
2176
1882
2808
951
1785
1672
1767
2283
1805
689
1274
2526
2938
2986
2211
656
81
2346
1455
2553
688
3004
3009
1806
1737
1721
2368
1514
2154
3029
2636
2345
1640
1994
1835
1278
1788
2255
316
2117
2637
1572
1939
1512
2385
2104
1454
2052
2199
1443
1135
2054
1764
2170
1230
1036
947
1766
2

In [26]:
tog.to_csv('tog.csv', index=False)

In [27]:
from rdflib import Graph, URIRef, RDF, Literal, Namespace
from rdflib.namespace import RDF
ex = Namespace('http://imdb.org/')
ont = Namespace('http://imdb.org/ontology/')
has_genre_uri = URIRef(ont+'has_genre')
director_uri = URIRef(ont+'has_director')
dir_mov_uri = URIRef(ont+'directed_by')
cast_uri = URIRef(ont+'has_actor')
cast_mov_uri = URIRef(ont+'acted_in')
published_uri = URIRef(ont+'published_in')
collab_uri = URIRef(ont+'collabs_with')
prefers_genre = URIRef(ont+'preferred_by')

dict = {}
test_train = []
rdf_graph = Graph()
grouped = tog.groupby('tconst')
id = 0 
for name, group in grouped:
    index = group.index[0]
    movie_id = name
    id += 1
    movie_uri = URIRef(ex+'mov'+str(movie_id))
    movie_title = group['originalTitle'][index]
    movie_genres = group['genres'][index]
    movie_year  = group['startYear'][index]
    year_uri = URIRef(ex+'year'+str(movie_year))

    rdf_graph.add((movie_uri, published_uri, year_uri))
    genre_uri = URIRef(ex+'gen'+movie_genres)
    test_train.append([movie_uri, id, genre_uri])
    print(name)
    print(group)
    for row in enumerate(group.index):
        print(row)
        if group.job[row[1]] == 'director':
            director_id = group['nconst'][row[1]]
            print(director_id)
            director_uri = URIRef(ex+'dir'+director_id)
            rdf_graph.add((movie_uri, dir_mov_uri, director_uri))
            rdf_graph.add((director_uri, dir_mov_uri, movie_uri))
            
        elif group.job[row[1]] == 'actor':
            actor_id = group['nconst'][row[1]]
            print(actor_id)
            actor_uri = URIRef(ex+'act'+actor_id)
            rdf_graph.add((movie_uri, cast_uri, actor_uri))
            rdf_graph.add((actor_uri, cast_mov_uri, movie_uri))
            rdf_graph.add((actor_uri, collab_uri, director_uri))
            rdf_graph.add((director_uri, collab_uri, actor_uri))
            rdf_graph.add((genre_uri, prefers_genre, actor_uri))
            rdf_graph.add((genre_uri, prefers_genre, director_uri))
    

tt0000009
        nconst  actor      primaryName  actress knownForTitles  director  \
726  nm0063086      0  Blanche Bayliss        1      tt0000009         0   
727  nm0085156      0  Alexander Black        0      tt0000009         1   

        tconst titleType originalTitle startYear  Drama  Comedy  Romance  \
726  tt0000009     movie    Miss Jerry      1894      0       0        1   
727  tt0000009     movie    Miss Jerry      1894      0       0        1   

     Thriller   genres       job  
726         0  Romance   actress  
727         0  Romance  director  
(0, 726)
(1, 727)
nm0085156
tt0000615
         nconst  actor      primaryName  actress knownForTitles  director  \
1504  nm0218953      0    Rhoda Dendron        1      tt0000615         0   
1505  nm0240418      1     William Duff        0      tt0000615         0   
1506  nm0346387      1     Arthur Guest        0      tt0000615         0   
1507  nm0581353      1  George Merriman        0      tt0000615         0   

   

In [28]:
rdf_graph.serialize('imdb_graph.nt', format='nt')

c:\Users\luisa\anaconda3\envs\aifb\lib\site-packages\rdflib\plugins\serializers\nt.py:35: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


<Graph identifier=N39d1ea37e40c459cb8e70cf9234d67ae (<class 'rdflib.graph.Graph'>)>

In [70]:
train = pd.DataFrame(test_train, columns=['movie', 'id','genre'])

from sklearn.model_selection import train_test_split
def stratified_train_test_split(train_test):
    train, test = train_test_split(train_test, test_size=0.04, random_state=42, stratify=train_test['genre'])
    return train, test
train, test = stratified_train_test_split(train)
test
train.to_csv('training_Set_new.csv', index=False)
test.to_csv('test_Set_new.csv', index=False)